# Train data preprocessing

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from tqdm.notebook import tqdm
import tsfresh
import sqlite3
import gc

In [2]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [3]:
le = preprocessing.LabelEncoder()

<IPython.core.display.Javascript object>

In [4]:
con = sqlite3.connect("../../data/amex-default-prediction/amex-data.sqlite")
con

<IPython.core.display.Javascript object>

In [5]:
customer_df = pd.read_sql_query("SELECT DISTINCT customer_ID FROM train_data", con)
customer_df

,customer_ID
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...
...,...
458908,ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...
458909,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...
458910,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...
458911,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...


<IPython.core.display.Javascript object>

In [6]:
chunks = np.array_split(
    customer_df["customer_ID"].values, int(len(customer_df) / 30000)
)  # chunk 30k
len(chunks)

15

<IPython.core.display.Javascript object>

In [7]:
def do_work(customer_ids):
    df = pd.read_sql_query(
        "SELECT * FROM train_data WHERE customer_ID IN ({seq})".format(
            seq="'" + "','".join(customer_ids) + "'"
        ),
        con,
    ).set_index("customer_ID")

    df.sort_values(["customer_ID", "S_2"], inplace=True)
    df.drop("S_2", axis=1, inplace=True)

    cat_columns = [
        "B_30",
        "B_38",
        "D_114",
        "D_116",
        "D_117",
        "D_120",
        "D_126",
        "D_63",
        "D_64",
        "D_66",
        "D_68",
    ]

    df[cat_columns] = df[cat_columns].astype(str)
    for cat_column in cat_columns:
        df[cat_column] = le.fit_transform(df[cat_column])

    categorical_df = df[cat_columns].groupby("customer_ID").max().copy()

    num_columns = list(set(df.columns) - set(cat_columns))

    df1 = (
        df[num_columns]
        .replace(r"^\s*$", np.nan, regex=True)
        .astype(float)
        .fillna(0)
        .copy()
    )
    df1 = df1.groupby("customer_ID").mean()

    label_df = pd.read_sql_query(
        "SELECT * FROM train_labels WHERE customer_ID IN ({seq})".format(
            seq="'" + "','".join(customer_ids) + "'"
        ),
        con,
    ).set_index("customer_ID")

    return df1.join([categorical_df, label_df])

<IPython.core.display.Javascript object>

In [8]:
for idx, customer_ids in enumerate(tqdm(chunks), 1):
    train_group_df = do_work(customer_ids)

    train_group_df.to_csv(
        "../../data/amex-default-prediction/train/train_group_data_{}.csv".format(idx)
    )

    gc.collect()

  0%|          | 0/15 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>